This is a simple notebook to do PCA on SDSS spectra and galaxy images.

It accompanies Chapter 7 of the book (2 of 4).

Copyright: Viviana Acquaviva (2023); see also other code credits below.

Modifications by Julieta Gruszko (2025)

License: [BSD-3-clause](https://opensource.org/license/bsd-3-clause/)

In [ ]:
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt

from sklearn import preprocessing, decomposition
import skimage
from skimage.transform import resize, rescale
from skimage import io

### Dimensionality Reduction

Principal Component Analysis (PCA) and similar algorithms are used for dimensionality reduction in data-intensive sciences. 

The main goal of linear PCA is to find the most representative linear combinations of features, so that each element of a data set can be expressed as the superposition (sum) of some salient vectors in feature space (they don't need to be elements of a data set. In the simplest linear PCA, the principal components are the eigenvectors of the covariance matrix of the data set.

If the number of components is very low (e.g. 2 or 3, PCA or other Dimensionality Reduction methods allow one to visualize a high-dimensional data set as a 2D or 3D plot. Scikit-learn has methods to compute PCA and several variants. Classic PCA has tough complexity: $\mathcal{O}[N^3].$ 

### Let's look at an example with galaxy spectra from 

https://ogrisel.github.io/scikit-learn.org/sklearn-tutorial/tutorial/astronomy/dimensionality_reduction.html#sdss-spectral-data

In [ ]:
data = np.load('../Data/spec4000_corrected.npz')

In [ ]:
wavelengths = data['wavelengths']
X = data['X']
y = data['y']
labels = data['labels'].astype('str')

In [ ]:
X.shape

In [ ]:
y

In [ ]:
labels #we don't really care about these; they are only used in the next cell to show some example spectra.

### We can plot some representative examples from each class, just to have a sense of what kind of spectra are in the data set.


In [ ]:
plt.figure(figsize=(10,5))

for i_class in (2, 3, 4, 5, 6):
    i = np.where(y == i_class)[0][0]
    l = plt.plot(wavelengths, X[i] + 20 * i_class)
    c = l[0].get_color()
    plt.text(6800, 2 + 20 * i_class, labels[i_class], color=c)

plt.subplots_adjust(hspace=0)
plt.xlabel('wavelength (Angstroms)')
plt.ylabel('flux + offset')
plt.title('Sample of Spectra');

### Question:
- How many instances are in our data set? How many features?
- What are the features of the data set?

We will try to represent our data with a variable amount of components.

In [ ]:
#  Perform PCA

scaler = preprocessing.StandardScaler() #It's important that data are centered!

Xn = scaler.fit_transform(X) #This is a standardization procedure.

pca_50 = decomposition.PCA(n_components=50, random_state=0)

pca_100 = decomposition.PCA(n_components=100, random_state=0)

pca_1000 = decomposition.PCA(n_components=1000, random_state=0)

X_proj_50 = pca_50.fit_transform(Xn) 

X_proj_100 = pca_100.fit_transform(Xn) 

X_proj_1000 = pca_1000.fit_transform(Xn)


### Questions:
- How many instances does $\texttt{X\_proj\_50}$ have? How many features?
- How many instances does $\texttt{X\_proj\_100}$ have? How many features?
- How many instances does $\texttt{X\_proj\_1000}$ have? How many features?
- Is $\texttt{X\_proj\_1000}$ identical to the original data set?

In [ ]:
#----------------------------------------------------------------------
#
#  plot PCA eigenspectra
#

plt.figure()

l = plt.plot(wavelengths, pca_50.mean_ - 0.15)
c = l[0].get_color()
plt.text(7000, -0.16, "mean", color=c) 

# In linear PCA, the first eigenvector is always the mean, 
# and the first n components are always the same

for i in range(4):
    
    l = plt.plot(wavelengths, pca_50.components_[i] + 0.15 * i)
    
    l = plt.plot(wavelengths, pca_100.components_[i] + 0.15 * i, linestyle = '-.')
    
    c = l[0].get_color()
    
    plt.text(7000, -0.01 + 0.15 * i, "component %i" % (i + 1), color=c)

    plt.ylim(-0.2, 0.6)
    
plt.xlabel('wavelength (Angstroms)')
plt.ylabel('scaled flux + offset')
plt.title('Mean Spectrum and Eigen-spectra')

plt.show()


We can estimate the contribution of each component by using the property "explained variance ratio".

These are simply the eigenvalues of the covariance matrix, and give an idea of how much each component contributes to the total variance in the data. Their cumulative sum (plotted a few cells below) gives the progressively increasing explained variance ratio for a given number of components.

In [ ]:
pca_50.explained_variance_ratio_ 

We can interpret the eigenvectors as the "basis" that explains most of the variability in the data. 

How can we know if this works? Let's reverse-engineer the process:

In [ ]:
Xrec_50 = pca_50.inverse_transform(X_proj_50) 

Xrec_100 = pca_100.inverse_transform(X_proj_100)

Xrec_1000 = pca_1000.inverse_transform(X_proj_1000) #This is useful for a sanity check; there should be no information loss, modulo numerical precision issues

Let's compare the original spectra to the results of inverting the PCA. Comment out/Uncomment lines below to look at each version of PCA.

In [ ]:
plt.figure(figsize=(10,5))

for i in range(4,8):
    plt.subplot(2,2,i-3)
    plt.plot(wavelengths, Xn[i], label = 'orig', c = 'k')
    plt.plot(wavelengths, Xrec_50[i], '--', label = 'new, 50 PCs', c = 'g')
    #plt.plot(wavelengths, Xrec_100[i], '--', label = 'new, 100 PCs', c = 'b')
    #plt.plot(wavelengths, Xrec_1000[i], '--', label = 'new, 1000 PCs', c = 'r')
    #plt.ylim(-0.5,0.5)
    plt.legend();
plt.xlabel('wavelength (Angstroms)');


It's difficult to see the differences from these plots, so let's look at residuals instead. Comment out/Uncomment lines below to look at each version of PCA.

In [ ]:
plt.figure(figsize=(10,5))

for i in range(4,8):
    plt.subplot(2,2,i-3)
    plt.plot(wavelengths, (Xrec_50[i]-Xn[i])/Xn[i], '--', label = '% diff 50', c = 'g')
    #plt.plot(wavelengths, (Xrec_100[i]-Xn[i])/Xn[i], '--', label = 'diff 100', c = 'b')
    #plt.plot(wavelengths, (Xrec_1000[i]-Xn[i])/Xn[i], '-.', label = 'diff 1000', c = 'k')
    plt.ylim(-0.5,0.5)
    plt.legend();
plt.xlabel('wavelength (Angstroms)');


Note that while many spectra are well represented by (for example) 50 components, some are not! 

### How can we know what's a good number of components?

To get an idea, we can plot the "explained_variance_ratio" property of the PCA decomposition. It looks a lot like the Elbow Method, but upside down; in particular, the variance explained by N components always increases with N, but there is usually a spot after which the returns tend to diminish. 

In [ ]:
plt.plot(np.cumsum(pca_1000.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');
plt.xlim(0,20)

### Question:
What would you recommend in the case above?

#### IMO, for science problems, how many components you need (and whether PCA is the right way to go about it) really depends on the scope. Unsupervised dimensionality reduction methods, such as PCA, are insensitive to the purpose, which is not always great. 
    
#### For example, in the case of the spectra above, while it is true that even a handful components capture a large fraction of the variance, the percent difference between the original spectra and the reconstructed ones can be large in the vicinity of "spiky" features, and also vary greatly from object to object. Whether this is acceptable for your particular science goal, only you can tell! 

### Let's now take a look at images.

This data set is composed by 200 images randomy selected from the Kaggle Galaxy Zoo challenge:

https://www.kaggle.com/c/galaxy-zoo-the-galaxy-challenge

The code below visualizes the first 25 objects in your data set. You can run it to get a view of the first 25 galaxies. Note: you might get an error message, in this case see here

https://stackoverflow.com/questions/43288550/iopub-data-rate-exceeded-in-jupyter-notebook-when-viewing-image

In [ ]:
#Takes < 1 minute, let's use non-bubbled images

images = []
for i in range(200):
    img =skimage.io.imread('../Data/Images/Image_'+str(i)+'.png')
    img_resized = resize(img,(100,100))
    length = np.prod(img_resized.shape)
    img_resized = np.reshape(img_resized,length)
    images.append(img_resized)
    
images = np.vstack(images)

In [ ]:
fig, axes = plt.subplots(ncols= 5, nrows = 5,figsize=(50,50))

ax = axes.ravel()

for i in range(ax.shape[0]):

    img = skimage.io.imread('../Data/Images/Image_'+str(i)+'.png')
    ax[i].imshow(img, cmap='gray')
    ax[i].set_xticks([])
    ax[i].set_yticks([])


### Here, we do the PCA decomposition on each of the RGB channels separately. I'm not sure of whether it is optimal.

In [ ]:
r_images = images.reshape(200, -1,  3)[:,:,0]

In [ ]:
r_images.shape

In [ ]:
#Calling PCA on images

estimator = decomposition.PCA(n_components=100)

r_images_PCA = estimator.fit_transform(r_images)

In [ ]:
#This tell us about the dimensionality reduction we have achieved.

r_images_PCA.shape

In [ ]:
components = estimator.components_

### We can plot the first 50 components.

In [ ]:
fig, axes = plt.subplots(5, 10, figsize=(12, 6),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i, ax in enumerate(axes.flat):
    ax.imshow((estimator.components_[i].reshape(100, 100)), cmap='bone')

### Question:
- From this plot, what would you guess as an optimal number of components?

We can use the explained variance ratio to see if there is an obvious optimal number of components.

In [ ]:
plt.plot(np.cumsum(estimator.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

### Question:
Note how different this plot is, compared to the case above, with the variance more equally distributed among many components. Approximately how many components would you need to retain ~ 90% of the variance?

### Let's now reconstruct the original images.

In [ ]:
r_projected = estimator.inverse_transform(r_images_PCA)

In [ ]:
# Plot the results
fig, ax = plt.subplots(2, 10, figsize=(15, 5),
                       subplot_kw={'xticks':[], 'yticks':[]},
                       gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i in range(10):
    ax[0, i].imshow(r_images[i].reshape(100, 100), cmap='gray')
    ax[1, i].imshow(r_projected[i].reshape(100, 100), cmap='gray')


We can do it for the three channels at once and then join them together:

In [ ]:
estimator = decomposition.PCA(n_components=100) 

r_images = images.reshape(200, -1,  3)[:,:,1]                     
estimator.fit(r_images)
r_images_PCA = estimator.fit_transform(r_images)
r_projected = estimator.inverse_transform(r_images_PCA)

g_images = images.reshape(200, -1,  3)[:,:,1]                     
estimator.fit(g_images)
g_images_PCA = estimator.fit_transform(g_images)
g_projected = estimator.inverse_transform(g_images_PCA)

b_images = images.reshape(200, -1,  3)[:,:,2]                     
estimator.fit(b_images)
b_images_PCA = estimator.fit_transform(b_images)
b_projected = estimator.inverse_transform(b_images_PCA)

In [ ]:
# Plot the results
fig, ax = plt.subplots(2, 5, figsize=(50, 20),
                       subplot_kw={'xticks':[], 'yticks':[]},
                       gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i in range(5):
    ax[0, i].imshow((np.dstack([r_images[i].reshape(100, 100)*255, g_images[i].reshape(100, 100)*255, 
        b_images[i].reshape(100,100)*255]).astype(np.uint8)))
    ax[1, i].imshow((np.dstack([r_projected[i].reshape(100, 100)*255, g_projected[i].reshape(100, 100)*255, 
        b_projected[i].reshape(100,100)*255]).astype(np.uint8)))

### Question:
What do you observe in these images? What part of the images is the PCA "prioritizing," from what you see?

## Conclusions

Dimensionality reduction techniques are useful both to build understanding of what's in the data and to make sizes more manageable.

Clustering and dimensionality reduction have a lot of overlap! See e.g. the Example # 2 of:

https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/05.11-K-Means.ipynb

You'll also see the combination in action on HW 4. 

#### Non-linear manifold techniques (t-SNE, Self Organized Maps...) are popular tools for visualization in 2D space, and they are useful for data exploration/investigation. However, they have tunable parameters that are not easy to tune, and they are hard to interpret.

See for example:

https://scikit-learn.org/stable/modules/manifold.html#manifold

or

https://www.superdatascience.com/blogs/the-ultimate-guide-to-self-organizing-maps-soms



### Acknowledgement Statement:

That's it for today, go ahead and upload this studio to Gradescope!